In [1]:
#總整理
import pandas as pd
import numpy as np
import json
import re
import pickle

In [5]:
path_train = './train_1_update.txt'
path_sample = './SampleData_deid.txt'
path_dev = './development_1.txt'

In [4]:
#對有標籤的 train資料進行整理 (誰講, 句子, mark_matrix)

def forsplit(s, arr):
    l1 = []            #一句話是由誰講的
    l2 = []            #講了什麼
    lis = s.split('：')
    arr0 = np.copy(arr[1:,:])   #註記矩陣
    for i in range(0, len(lis)):
        if i == 0:
            l1.append(lis[i] + '：')
        elif i == len(lis) - 1:
            l2.append(lis[i])
        else:
            l2.append(lis[i][0:-2])
            x = lis[i][-2:]
            l1.append(x + '：')
    
    r = arr0.shape[0]        #共幾串文字需要註記
    w = len(l1[0])           #紀錄現在的字元總數
    s = 0                    #紀錄正要設定的註記位置
    for i in range(len(l2)):
        if s == r:
            break
        for j in range(s, r):
            if int(arr0[j, 1]) <= w + len(l2[i]):
                arr0[j, 0] = i
                #print('w=', w, '  int=', int(arr0[j, 1]))
                arr0[j, 1] = int(arr0[j, 1]) - w
                arr0[j, 2] = int(arr0[j, 2]) - w
                s += 1
            elif int(arr0[j, 1]) > w + len(l2[i]):
                try:
                    w += len(l2[i]) + len(l1[i+1])
                except:
                    print('八成是 i + 1 超過了')
                break
                
    return l1, l2, arr0

def dataprocess(path):
    datalist=[]
    with open(path, 'r', encoding="utf-8") as f:
        x = ''
        for line in f:
            if line != '--------------------\n':
                x = x + line + '\n'
            else:
                datalist.append(x)
                x = ''
    datadir = {}
    for i, j in enumerate(datalist):
        c = []
        c1 = j.split('\n')
        for k in c1:
            if k:
                c.append(k)
                datadir[i] = c
    #return datadir
    
    datadir1 = {}
    for i, j in datadir.items():
        f = []
        for k1, k2 in enumerate(j):
            if k1 == 0:
                c1 = k2
            else:
                f.append(k2)
        f1 = []
        for l in f:
            f1.append(l.split('\t'))
        y = np.array(f1)        
        datadir1[i] = (c1, y)
    
    datadir2 = {}
    for i, j in datadir1.items():
        x1, x2, x3 = forsplit(j[0], j[1])
        datadir2[i] = (x1, x2, x3)
        
    return datadir2

In [6]:
train_data = dataprocess(path_train)
sample_data = dataprocess(path_sample)

In [9]:
#對沒有標籤的 train資料進行整理 (誰講, 句子)
def forsplit2(s):
    l1 = []            #一句話是由誰講的
    l2 = []            #講了什麼
    lis = s.split('：')
    for i in range(0, len(lis)):
        if i == 0:
            l1.append(lis[i] + '：')
        elif i == len(lis) - 1:
            l2.append(lis[i])
        else:
            l2.append(lis[i][0:-2])
            x = lis[i][-2:]
            l1.append(x + '：')
                
    return l1, l2

def dataprocess_no_matrix(path):
    datalist=[]
    with open(path, 'r', encoding="utf-8") as f:
        x = ''
        for line in f:
            if line != '--------------------\n':
                x = x + line + '\n'
            else:
                datalist.append(x)
                x = ''
                
    datadir = {}
    for i, j in enumerate(datalist):
        c = []
        c1 = j.split('\n')
        for k in c1:
            if k:
                c.append(k)
                datadir[i] = c
    #return datadir
    
    datadir1 = {}
    for i, j in datadir.items():
        for k1, k2 in enumerate(j):
            if k1 == 0:
                c1 = k2
            else:
                datadir1[i] = k2
    
    datadir2 = {}
    for i, j in datadir1.items():
        x1, x2 = forsplit2(j)
        datadir2[i] = (x1, x2)
        
    return datadir2

In [10]:
dev_data = dataprocess_no_matrix(path_dev)

In [12]:
#train, sample 合併
train_sample_data = {}
c = 0
for i, j in train_data.items():
    train_sample_data[i] = j
    c += 1

for i, j in sample_data.items():
    train_sample_data[c + i] = j

In [13]:
#計算 label 有哪些

labels = []
for _, i in train_sample_data.items():
    for j in range(i[2].shape[0]):
        if i[2][j, 4] not in labels:
            labels.append(i[2][j, 4])

In [14]:
labels

['time',
 'med_exam',
 'profession',
 'name',
 'location',
 'family',
 'ID',
 'clinical_event',
 'education',
 'money',
 'contact',
 'organization']

In [15]:
#每筆資料的句子，換成 list 存字
sentence_dic = {}
for i, j in train_sample_data.items():
    sentence_word = []
    for sen in j[1]:
        s = ''
        for w in sen:
            s = s + w
        sentence_word.append(s)
    sentence_dic[i] = sentence_word
    

In [ ]:
#句子的字標記
mark_dic = {}        #馬卡夫
for i, j in data.items():
    mark = []
    for n in range(len(j[1])):
        s = ['0' for _ in range(len(j[1][n]))]
        mark.append(s)
    mark_dic[i] = mark

for i ,j in mark_dic.items():
    m = data[i][2]
    for k in range(m.shape[0]):
        v = m[k, :]
        sen = j[int(v[0])]
        sen[int(v[1])] = 'B-' + v[4][0:3]
        for l in range(int(v[1])+1, int(v[2])):
            sen[l] = 'I-' + v[4][0:3]
        

In [ ]:
# 整合資料
bind = []
bind_mark = []
for i0, i in bind_train.items():
    data_i = []
    bind_i = []
    for j in range(len(i[0])):
        data_i0 = []
        bind_i0 = []
        for w in i[0][j]:
            data_i0.append(w)
            bind_i0.append('O')
        for w in i[1][j]:
            data_i0.append(w)
            bind_i0.append('O')
        data_i.append(data_i0)
        bind_i.append(bind_i0)
    bind.append(data_i)
    bind_mark.append(bind_i)
    
len(bind)

In [ ]:
for i0, i in bind_train.items():
    m = i[2]
    for row in range(m.shape[0]):
        r = m[row, :]
        for p in range(int(r[1]), int(r[2])):
            if p == int(r[1]):
                bind_mark[i0][int(r[0])][p+3] = 'B-{}'.format(r[4])
            else:
                bind_mark[i0][int(r[0])][p+3] = 'I-{}'.format(r[4])


In [17]:
with open('./train_sample_data.pickle', 'wb') as f:
    pickle.dump(train_sample_data, f)
with open('./dev_data.pickle', 'wb') as f:
    pickle.dump(dev_data, f)